In [20]:
# Define the gpu  on the gpu machine
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=6

import pandas as pd
import json
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np
from collections import Counter

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=6


Calculate token distribution - vector of values per tokens (how many times does each token occur) - create a dictionary with all tokens from train set and specific test set, iterate through the tokens and count how many times each ocurrs. Calculate cosine similarity.

Do this on label level as well to see whether this explains good performance on some of the labels for Maltese.

## Tokenize and count tokens for train_df

Code for tokenization (it is now already done):

In [18]:
# Import the train dataset
train = load_dataset("TajaKuzman/X-GENRE-multilingual-text-genre-dataset", "train")

# To open as Pandas DataFrame:
train_df = pd.DataFrame(train["train"])

display(train_df.head(2))

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the train dataset
tokens_train = []
integers_train = []
token_list_train = []

for text in tqdm(train_df.text.to_list()):
	encoded_text = tokenizer(text)
	# Take all tokens_train, except the beginning (<s>) and end (</s>) token
	current_tokens_train = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
	tokens_train.append(current_tokens_train)
	token_list_train.extend(current_tokens_train)
	integers_train.append(encoded_text.input_ids[1:-1])

train_df["tokens_train"] = tokens_train
train_df["token_ids"] = integers_train

#print(token_list_train[:10])
#print(len(token_list_train))

# Create a list of tokens, where we take only the first 512 tokens
train_tokens_shortened = []

for i in train_df["tokens_train"].to_list():
	train_tokens_shortened.extend(i[:512])

print(len(train_tokens_shortened))

train_df.head(3)

# Save the tokenized version
train_df.to_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

,text,labels,dataset,language
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English


100%|██████████| 1772/1772 [00:09<00:00, 189.74it/s]


699465


Code to count tokens:

In [2]:
# Open the tokenized df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51..."


In [11]:
# Create a dictionary that counts all the token occurrences

# Create a list of tokens, where we take only the first 512 tokens
train_tokens_shortened = []

for i in train_df["tokens_train"].to_list():
	train_tokens_shortened.extend(i[:512])

print(len(train_tokens_shortened))

# Create a dictionary which counts the occurrences of the words

word_dict_train = Counter(train_tokens_shortened)

# Sort the dictionary alphabetically based on keys
word_dict_train = dict(sorted(word_dict_train.items()))

print(list(word_dict_train.items())[:100])
print(len(word_dict_train))

699465
[('!', 430), ('!!', 23), ('!!!', 14), ('!!!!', 6), ('!!!!!', 1), ('!!!!!!', 1), ('!!!!!!!', 1), ('!"', 14), ('!)', 10), ('!),', 1), ('!).', 2), ('"', 528), ('")', 7), ('"),', 6), ('").', 7), ('",', 56), ('".', 83), ('"...', 3), ('";', 3), ('"?', 6), ('#', 4), ('$', 8), ('%', 4), ('%)', 1), ('&', 46), ("'", 4517), ('(', 31), ('(1', 1), (')', 788), ('),', 242), (').', 297), ('):', 13), (');', 5), ('*', 19), ('**', 2), ('****', 1), ('+', 15), ('+5', 1), (',', 23447), (',«', 12), ('-', 2803), ('---', 3), ('------', 5), ('----------------', 41), ('-0', 3), ('-01', 1), ('-02', 2), ('-02-', 2), ('-03-', 4), ('-06', 1), ('-06-', 3), ('-09-', 1), ('-1', 15), ('-1)', 3), ('-10', 2), ('-10-', 1), ('-11', 6), ('-11-', 1), ('-12', 8), ('-13', 6), ('-14', 3), ('-15', 7), ('-16', 9), ('-17', 3), ('-18', 9), ('-19', 5), ('-2', 10), ('-20', 3), ('-2000', 3), ('-2005', 1), ('-2007', 8), ('-2009', 1), ('-2010', 1), ('-2011', 1), ('-2012', 1), ('-2014', 1), ('-2020', 1), ('-21', 4), ('-22', 3), ('-

The train dataset has 699.465 tokens and 27.025 unique words.

In [12]:
# See the most frequent tokens:

# Sort the dictionary by values (word counts) in descending order
sorted(word_dict_train.items(), key=lambda x: x[1], reverse=True)[:10]


[(',', 23447),
 ('.', 21407),
 ('▁', 19553),
 ('▁the', 18860),
 ('s', 14184),
 ('▁to', 10762),
 ('▁of', 9912),
 ('▁and', 9752),
 ('▁in', 9140),
 ('▁a', 8341)]

In [14]:
# Save the dictionary of tokens
with open("datasets/tokenized_datasets/X-GENRE-train-token-count.json", "w") as train_count_file:
	json.dump(word_dict_train, train_count_file)

# Tokenize and count tokens for test sets

Code with which I tokenized the datasets:

In [15]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [ ]:
# Define an array of token overlap
token_overlap_results = {}

# Loop through the datasets and calculate token overlap
for lang in list(main_dict.keys()):
	df = pd.DataFrame(main_dict[lang]["dataset"])

	tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

	tokens = []
	integers = []
	token_list = []

	print("Tokenizing text.")

	for text in tqdm(df.text.to_list()):
		encoded_text = tokenizer(text)
		# Take all tokens, except the beginning (<s>) and end (</s>) token
		current_tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
		# Shorten the list to 512, as tokens after that were not observed by the classifier
		current_tokens = current_tokens[:512]
		tokens.append(current_tokens)
		token_list.extend(current_tokens)
		integers.append(encoded_text.input_ids[1:-1][:512])

	df["tokens"] = tokens
	df["token_ids"] = integers

	print(token_list[:10])
	print("All tokens:")
	print(len(token_list))

	# See how many tokens overlap
	overlap_counter = 0

	print("Calculating overlap.")

	# Save tokens that overlap for further inspection
	overlap_token_list = []

	#overlap_counter = sum(1 for element in token_list if element in train_tokens_shortened)
	for token in tqdm(token_list):
		if token in train_tokens_shortened:
			overlap_counter += 1
			overlap_token_list.append(token)

	# Out of all tokens, how many overlap?
	overlap_per = overlap_counter/len(token_list)

	print(f"Number of tokens that overlap: {overlap_counter}")
	print(f"Percentage of overlap: {overlap_per}")

	# Update the dataset in the dictionary
	main_dict[lang]["dataset"] = df.to_dict()

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"] = {"overlap_percentage":overlap_per, "token_list": token_list, "overlap_token_list":overlap_token_list}

	# Add to the results
	token_overlap_results[lang] = {"percentage": overlap_per, "overlap_list": overlap_token_list}

	# Convert tokens back to words
	#print(tokenizer.convert_tokens_to_string(tokens))

# Inspect the results
overlap_df = pd.DataFrame(token_overlap_results)
	

In [26]:
overlap_df = pd.DataFrame(token_overlap_results).transpose()
overlap_df

,percentage,overlap_list
mt,0.817085,"[▁Angel, o, ▁Che, t, ,, ▁se, ▁j, kun, d, u, ▁p..."
el,0.161428,"[asi, asi, ,, ,, ▁Re, ception, ., ▁driver, ▁es..."
tr,0.521502,"[▁A, L, ▁Der, s, i, ▁ve, ▁Beli, r, leme, ▁S, h..."
sq,0.605775,"[▁Blog, ▁“, U, ▁kam, ▁me, jet, .”, ▁Jer, ▁31, ..."
is,0.517575,"[▁[, is, ], ▁er, fi, ▁reg, ▁sett, ar, lag, sin..."
uk,0.156675,"[., ,, ,, ?, ▁-, ., ?, ▁-, ', ., ,, ., ▁, ▁(, ..."
ca,0.744881,"[▁P, à, gine, s, ▁En, nada, ▁Porto, ▁uns, ▁die..."
mk,0.145989,"[,, ▁T, CL, ,, ▁T, CL, :, ▁Alca, tel, ▁Mobile,..."
hr,0.821517,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, ,, ▁za, ▁pa..."
sl,0.974289,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,..."


In [36]:
for token in overlap_df["overlap_list"].to_list()[0][:2]:
	if len(token) > 1:
		non_short_counter += 1

▁Angel


In [37]:
# Inspect how many tokens are one character long

counters = []
pers = []

for token_list_el in overlap_df["overlap_list"].to_list():
	non_short_counter = 0
	for token in token_list_el:
		if len(token) > 1:
			non_short_counter += 1
	
	counters.append(non_short_counter)
	pers.append(non_short_counter/len(token_list_el))

# Add to df
overlap_df["non_short"] = counters

# Calculate the percentage of overlap tokens that have more than 1 character
overlap_df["non_short_per"] = pers

overlap_df


,percentage,overlap_list,non_short,non_short_per
mt,0.817085,"[▁Angel, o, ▁Che, t, ,, ▁se, ▁j, kun, d, u, ▁p...",23083,0.723628
el,0.161428,"[asi, asi, ,, ,, ▁Re, ception, ., ▁driver, ▁es...",1883,0.373389
tr,0.521502,"[▁A, L, ▁Der, s, i, ▁ve, ▁Beli, r, leme, ▁S, h...",11141,0.722269
sq,0.605775,"[▁Blog, ▁“, U, ▁kam, ▁me, jet, .”, ▁Jer, ▁31, ...",12459,0.768315
is,0.517575,"[▁[, is, ], ▁er, fi, ▁reg, ▁sett, ar, lag, sin...",10915,0.711399
uk,0.156675,"[., ,, ,, ?, ▁-, ., ?, ▁-, ', ., ,, ., ▁, ▁(, ...",1258,0.253476
ca,0.744881,"[▁P, à, gine, s, ▁En, nada, ▁Porto, ▁uns, ▁die...",15657,0.763123
mk,0.145989,"[,, ▁T, CL, ,, ▁T, CL, :, ▁Alca, tel, ▁Mobile,...",1270,0.314746
hr,0.821517,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, ,, ▁za, ▁pa...",17678,0.810620
sl,0.974289,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...",21567,0.841935


In [49]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

Add token counts information

In [23]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [25]:
token_number = {}
type_number = {}

for lang in list(main_dict.keys()):
	print(f"Creating token dict for {lang}")
	current_token_list = main_dict[lang]["token_overlap"]["token_list"]

	print(f"No of tokens: {len(current_token_list)}")

	# Create a dictionary which counts the occurrences of the tokens

	word_dict_test = Counter(current_token_list)

	# Sort the dictionary alphabetically based on keys
	word_dict_test = dict(sorted(word_dict_test.items()))

	# Add information on no. of tokens and words to a dict
	token_number[lang] = len(current_token_list)
	type_number[lang] = len(word_dict_test)

	print(list(word_dict_test.items())[:100])
	print(f"No of unique tokens: {len(word_dict_test)}")

	# Add the count of tokens to the main dictionary
	main_dict[lang]["token_overlap"]["token_count"] = word_dict_test

# Create a dataframe for statistics
token_df = pd.DataFrame({"tokens": token_number, "types": type_number})
print(token_df.to_markdown(index=False))

Creating token dict for mt
No of tokens: 39040
[('!', 9), ('"', 34), ('",', 11), ('".', 6), ('#', 20), ("'", 462), ('(', 3), ('(1)', 4), ('(2)', 4), ('(3)', 4), (')', 50), ('),', 15), (').', 15), (');', 1), (',', 884), ('-', 2119), ('-0', 1), ('-1', 1), ('-11', 1), ('-12-', 2), ('-16', 5), ('-17', 1), ('-18', 2), ('-19', 1), ('-2', 5), ('-20', 1), ('-200', 1), ('-2000', 1), ('-2007', 1), ('-2009', 1), ('-2014', 1), ('-2015', 1), ('-2016', 2), ('-2017', 2), ('-21', 1), ('-22', 1), ('-23', 1), ('-24', 2), ('-28', 1), ('-30', 4), ('-35', 1), ('-4', 1), ('-5', 1), ('-500', 1), ('-6', 1), ('-7', 1), ('-90', 2), ('.', 724), ('."', 5), ('...', 1), ('.”', 3), ('/', 11), ('/08', 3), ('/13', 1), ('/14', 3), ('/19', 4), ('/24', 2), ('/3', 6), ('/4', 2), ('/5', 2), ('/7', 2), ('0.7', 1), ('00', 1), ('016', 1), ('02', 2), ('04', 2), ('05', 1), ('050', 1), ('09.', 2), ('1', 5), ('100', 1), ('112', 1), ('12', 2), ('135', 2), ('14', 1), ('152', 1), ('164', 1), ('19', 5), ('1962', 1), ('1998', 1), ('2'

In [26]:
print(token_df.to_markdown())

|    |   tokens |   types |
|:---|---------:|--------:|
| mt |    39040 |    4787 |
| el |    31240 |    4751 |
| tr |    29578 |    6272 |
| sq |    26769 |    4891 |
| is |    29644 |    4615 |
| uk |    31677 |    6507 |
| ca |    27544 |    5314 |
| mk |    27639 |    5468 |
| hr |    26546 |    6222 |
| sl |    26292 |    5763 |


In [35]:
# Let's see the most frequent tokens
most_frequent = {}
# Sort the dictionary by values (word counts) in descending order
for lang in list(main_dict.keys()):
	most_frequent[lang] = (sorted(main_dict[lang]['token_overlap']['token_count'].items(), key=lambda x: x[1], reverse=True)[:10])

print(pd.DataFrame({"most_frequent_type": most_frequent}).to_markdown())

|    | most_frequent_type                                                                                                                       |
|:---|:-----------------------------------------------------------------------------------------------------------------------------------------|
| ca | [(',', 1080), ('▁de', 1014), ('.', 675), ('s', 648), ('▁i', 564), ('▁la', 559), ('▁a', 529), ('▁que', 438), ("'", 360), ('’', 334)]      |
| el | [('▁', 1017), ('.', 801), (',', 782), ('▁και', 553), ('ς', 525), ('▁να', 351), ('▁το', 321), ('▁του', 292), ('▁την', 268), ('▁με', 264)] |
| hr | [(',', 878), ('.', 766), ('▁i', 546), ('▁u', 430), ('a', 413), ('▁je', 350), ('▁na', 282), ('▁za', 253), ('▁se', 239), ('e', 219)]       |
| is | [('.', 1017), ('▁og', 628), ('▁að', 613), (',', 600), ('▁', 528), ('▁í', 434), ('▁á', 388), ('▁er', 357), ('s', 326), ('▁sem', 279)]     |
| mk | [(',', 1021), ('▁на', 983), ('.', 738), ('▁и', 619), ('▁за', 475), ('▁да', 378), ('▁во', 376), ('▁се', 365), ('▁', 34

In [36]:
# Save the main dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

# Compare train df and test set overlap

Create a dictionary with all tokens from train set and specific test set, iterate through the tokens and count how many times each ocurrs. Calculate cosine similarity.

Do this on label level as well to see whether this explains good performance on some of the labels for Maltese.

Cosine similarity is a metric used to measure the similarity of two vectors. Specifically, it measures the similarity in the direction or orientation of the vectors ignoring differences in their magnitude or scale. Both vectors need to be part of the same inner product space, meaning they must produce a scalar through inner product multiplication. The similarity of two vectors is measured by the cosine of the angle between them. The similarity can take values between -1 and +1. Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity. 

Cosine similarity ignores 0-0 matches. Counting 0-0 matches in sparse data would inflate similarity scores. Another commonly used metric that ignores 0-0 matches is Jaccard Similarity.

In [2]:
def cosine_similarity(x, y):
    
    # Ensure length of x and y are the same
    if len(x) != len(y) :
        return None
    
    # Compute the dot product between x and y
    dot_product = np.dot(x, y)
    
    # Compute the L2 norms (magnitudes) of x and y
    magnitude_x = np.sqrt(np.sum(x**2)) 
    magnitude_y = np.sqrt(np.sum(y**2))
    
    # Compute the cosine similarity
    cosine_similarity = dot_product / (magnitude_x * magnitude_y)
    
    return cosine_similarity

In [3]:
# Import train token count
# Save the dictionary of tokens
with open("datasets/tokenized_datasets/X-GENRE-train-token-count.json", "r") as train_count_file:
	train_count = json.load(train_count_file)

list(train_count.items())[:5]

[('!', 430), ('!!', 23), ('!!!', 14), ('!!!!', 6), ('!!!!!', 1)]

In [4]:
# Import the main dict for test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "r") as file:
	main_dict = json.load(file)

list(main_dict["sl"]["token_overlap"]["token_count"].items())[:10]

[('!', 30),
 ('!!', 1),
 ('!"', 1),
 ('!),', 1),
 ('"', 11),
 ('".', 2),
 ('"...', 1),
 ('&', 8),
 ("'", 3),
 ("''", 1)]

In [8]:
cosine_sim = {}
vector_size = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")
	# Get token count for current lang
	current_lang_count = main_dict[lang]["token_overlap"]["token_count"]

	print(f"Number of token types for {lang}: {len(list(current_lang_count.keys()))}")

	# For each test set, create a vector of token counts. Take only tokens that are present either in train_df or test set.
	intersection_dict = {}

	# First, create a list of tokens that are present in either one or the other list
	intersection_keys = []
	intersection_keys.extend(list(current_lang_count.keys()))
	intersection_keys.extend(list(train_count.keys()))
	# Remove duplicated keys
	intersection_keys = list(set(intersection_keys))
	print(f"Number of intersecting types: {len(intersection_keys)}")

	# Then create a dictionary for 1) train df and 2) test df with counts of token types that occur in either of the datasets
	train_intersect_dict = {}
	test_intersect_dict = {}

	for i in intersection_keys:
		try:
			train_intersect_dict[i] = train_count[i]
		except:
			train_intersect_dict[i] = 0
		try:
			test_intersect_dict[i] = current_lang_count[i]
		except:
			test_intersect_dict[i] = 0


	# Create a df with intersecting keys
	intersect_df = pd.DataFrame({"train": train_intersect_dict, "test": test_intersect_dict})

	display(intersect_df.head(10))

	# Calculate cosine similarity
	current_cosine_sim = cosine_similarity(np.array(intersect_df["train"].to_list()), np.array(intersect_df["test"].to_list()))

	print(f"Cosine similarity for {lang}: {current_cosine_sim}")

	# Add to the main dictionary
	main_dict[lang]["token_overlap"]["cosine_similarity"] = current_cosine_sim
	main_dict[lang]["token_overlap"]["intersection_df"] = intersect_df.to_dict()
	main_dict[lang]["token_overlap"]["intersection_vector_size"] = len(intersection_keys)

	# Add to a dict of results
	cosine_sim[lang] = current_cosine_sim
	vector_size[lang] = len(intersection_keys)

Processing mt
Number of token types for mt: 4787
Number of intersecting types: 28226


,train,test
▁boot,11,0
▁Trend,2,0
set,47,1
▁skills,38,0
▁ER,2,0
▁Install,1,1
▁Ż,0,7
▁Bran,9,0
NJE,3,0
▁primordial,1,0


Cosine similarity for mt: 0.4142476865826809
Processing el
Number of token types for el: 4751
Number of intersecting types: 30954


,train,test
▁Απριλίου,0,3
▁boot,11,0
▁Trend,2,0
set,47,0
▁skills,38,0
▁ER,2,0
▁Install,1,0
έλθει,0,1
▁Bran,9,0
NJE,3,0


Cosine similarity for el: 0.5273207128330722
Processing tr
Number of token types for tr: 6272
Number of intersecting types: 30846


...

Cosine similarity for mk: 0.4225320009293607
Processing hr
Number of token types for hr: 6222
Number of intersecting types: 28864


,train,test
▁boot,11,0
▁Essen,0,2
▁Trend,2,0
set,47,0
▁skills,38,0
▁ER,2,0
▁Install,1,0
▁Bran,9,0
NJE,3,0
▁primordial,1,0


Cosine similarity for hr: 0.5662098277228971
Processing sl
Number of token types for sl: 5763
Number of intersecting types: 27507


,train,test
▁boot,11,0
▁Trend,2,0
set,47,0
▁skills,38,0
▁ER,2,0
▁Install,1,0
▁Bran,9,0
NJE,3,1
▁primordial,1,0
TION,18,0


Cosine similarity for sl: 0.6334513674796362


In [12]:
# Show results
cosine_sim_df = pd.DataFrame({"cosine_similarity": cosine_sim, "vector_size": vector_size}).sort_values(by="cosine_similarity", ascending=False)
print(cosine_sim_df.to_markdown())

|    |   cosine_similarity |   vector_size |
|:---|--------------------:|--------------:|
| sl |            0.633451 |         27507 |
| tr |            0.593847 |         30846 |
| uk |            0.584612 |         33121 |
| hr |            0.56621  |         28864 |
| el |            0.527321 |         30954 |
| ca |            0.525439 |         29443 |
| is |            0.525135 |         29518 |
| sq |            0.434489 |         29168 |
| mk |            0.422532 |         31837 |
| mt |            0.414248 |         28226 |


In [19]:
main_dict["sl"]["token_overlap"].keys()

dict_keys(['overlap_percentage', 'token_list', 'overlap_token_list', 'token_count', 'cosine_similarity', 'intersection_df', 'intersection_vector_size'])

In [18]:
# Save the main dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

# Compare token overlap on label level 

# Compare train df and test set overlap

Separate texts in train and test sets into genre datasets. Create a dictionary with all tokens from train set and specific test set for each genre dataset for each language, iterate through the tokens and count how many times each ocurrs. Calculate cosine similarity.

In [21]:
def cosine_similarity(x, y):
    
    # Ensure length of x and y are the same
    if len(x) != len(y) :
        return None
    
    # Compute the dot product between x and y
    dot_product = np.dot(x, y)
    
    # Compute the L2 norms (magnitudes) of x and y
    magnitude_x = np.sqrt(np.sum(x**2)) 
    magnitude_y = np.sqrt(np.sum(y**2))
    
    # Compute the cosine similarity
    cosine_similarity = dot_product / (magnitude_x * magnitude_y)
    
    return cosine_similarity

### Create label-level token counts for train dataset

In [22]:
# Open the tokenized train df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51..."


In [23]:
train_df.labels.unique()

array(['Other', 'Information/Explanation', 'News', 'Instruction',
       'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal',
       'Promotion'], dtype=object)

In [33]:
# Separate the train df into label-based dfs

# Create a dictionary of token counts that are label based
label_token_count_train = {}
token_count = {}
type_count = {}

for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
	print(f"Processing {label}")

	label_df = train_df[train_df["labels"] == label]
	display(label_df.head(3))

	# Create a dictionary that counts all the token occurrences

	# Create a list of tokens, where we take only the first 512 tokens
	train_tokens_shortened = []

	for i in label_df["tokens_train"].to_list():
		train_tokens_shortened.extend(i[:512])

	print(f"Number of all tokens: {len(train_tokens_shortened)}")

	# Create a dictionary which counts the occurrences of the words

	word_dict_train = Counter(train_tokens_shortened)

	# Sort the dictionary alphabetically based on keys
	word_dict_train = dict(sorted(word_dict_train.items()))

	print(list(word_dict_train.items())[:20])
	print(f"Number of all types: {len(list(word_dict_train.keys()))}")

	# Add to dictionaries
	label_token_count_train[label] = word_dict_train
	token_count[label] = len(train_tokens_shortened)
	type_count[label] = len(list(word_dict_train.keys()))



Processing Information/Explanation


,text,labels,dataset,language,tokens_train,token_ids
2,Abstract Objective: Reporting bias due to soci...,Information/Explanation,CORE,English,"[▁Abstract, ▁Object, ive, :, ▁Report, ing, ▁bi...","[233973, 134549, 5844, 12, 34798, 214, 333, 16..."
3,In 2009 the song was the focus of a successful...,Information/Explanation,CORE,English,"[▁In, ▁2009, ▁the, ▁song, ▁was, ▁the, ▁focus, ...","[360, 1877, 70, 11531, 509, 70, 32153, 111, 10..."
39,Story: Whaling Page 4 -- M?ori and whaling Wha...,Information/Explanation,CORE,English,"[▁Story, :, ▁W, hal, ing, ▁Page, ▁4, ▁--, ▁M, ...","[30575, 12, 601, 4200, 214, 14231, 201, 4210, ..."


Number of all tokens: 124130
[('!', 17), ('!"', 1), ('"', 72), ('")', 1), ('").', 2), ('",', 8), ('".', 15), ('"?', 1), ('%', 3), ('&', 14), ("'", 393), ('(', 4), ('(1', 1), (')', 138), ('),', 64), (').', 80), ('):', 1), ('*', 2), (',', 4231), (',«', 2)]
Number of all types: 14678
Processing News


,text,labels,dataset,language,tokens_train,token_ids
4,QuotW This was the week when neither rumours o...,News,CORE,English,"[▁Quo, t, W, ▁This, ▁was, ▁the, ▁week, ▁when, ...","[43851, 18, 1456, 3293, 509, 70, 5895, 3229, 2..."
5,KaZaA claims it can't stop users sharing music...,News,CORE,English,"[▁Ka, Za, A, ▁claims, ▁it, ▁can, ', t, ▁stop, ...","[1136, 16737, 284, 140526, 442, 831, 25, 18, 7..."
9,Nebraska fans checking out airfare for a trip ...,News,CORE,English,"[▁Ne, bra, ska, ▁fans, ▁checking, ▁out, ▁air, ...","[799, 2844, 937, 35992, 175199, 1810, 1831, 44..."


Number of all tokens: 136557
[('!', 38), ('!!!', 1), ('!"', 1), ('"', 177), ('").', 1), ('",', 21), ('".', 25), ('";', 1), ('"?', 1), ('#', 1), ('$', 3), ('&', 2), ("'", 968), ('(', 7), (')', 124), ('),', 47), (').', 31), ('+', 7), ('+5', 1), (',', 4559)]
Number of all types: 15319
Processing Instruction


...

Number of all tokens: 46860
[('!', 22), ('!!', 1), ('!"', 5), ('!)', 1), ('!),', 1), ('"', 34), ('";', 1), ('#', 1), ('&', 2), ("'", 542), (')', 42), ('),', 3), (').', 5), ('*', 1), ('+', 1), (',', 1982), ('-', 262), ('------', 2), ('----------------', 16), ('-09-', 1)]
Number of all types: 5990
Processing Legal


,text,labels,dataset,language,tokens_train,token_ids
35,Full Terms and Conditions Eligibility to enter...,Legal,CORE,English,"[▁Full, ▁Terms, ▁and, ▁Condi, tions, ▁E, ligi,...","[9312, 165504, 136, 46347, 5256, 241, 7883, 83..."
110,Commonwealth Consolidated Acts INCOME TAX ASSE...,Legal,CORE,English,"[▁Common, we, al, th, ▁Con, solid, ated, ▁Act,...","[151301, 1177, 289, 927, 1657, 97281, 27686, 2..."
122,"This Terms of Use Agreement (""Agreement"") is b...",Legal,CORE,English,"[▁This, ▁Terms, ▁of, ▁Use, ▁Agreement, ▁("", A,...","[3293, 165504, 111, 36836, 186670, 24073, 284,..."


Number of all tokens: 28496
[('"', 7), ('")', 4), ('"),', 1), ('").', 1), ('".', 2), ("'", 47), (')', 61), ('),', 10), (').', 11), (');', 2), (',', 833), ('-', 66), ('----------------', 20), ('-1', 5), ('-2', 1), ('-2000', 2), ('-21', 1), ('-30', 1), ('-31', 1), ('-5', 1)]
Number of all types: 4425
Processing Promotion


,text,labels,dataset,language,tokens_train,token_ids
79,Post navigation Citizen Fish are back on the r...,Promotion,CORE,English,"[▁Post, ▁navigation, ▁Citizen, ▁Fish, ▁are, ▁b...","[2795, 134470, 193223, 104796, 621, 4420, 98, ..."
130,Win yourself a FREE copy of the BradyGames off...,Promotion,CORE,English,"[▁Win, ▁yourself, ▁a, ▁FREE, ▁copy, ▁of, ▁the,...","[17686, 31949, 10, 86697, 43658, 111, 70, 5859..."
179,Do You Want To Know The Quick Secret for On Pa...,Promotion,CORE,English,"[▁Do, ▁You, ▁Want, ▁To, ▁Know, ▁The, ▁Quick, ▁...","[984, 2583, 42335, 717, 70829, 581, 89038, 390..."


Number of all tokens: 88626
[('!', 51), ('!!!', 1), ('!).', 1), ('"', 10), ('"),', 1), ('",', 1), ('#', 1), ('&', 1), ("'", 235), (')', 34), ('),', 10), (').', 21), ('*', 1), (',', 2782), ('-', 352), ('-02', 2), ('-1', 2), ('-12', 5), ('-14', 2), ('-16', 1)]
Number of all types: 12548


In [35]:
# Create df with results
label_results_train = pd.DataFrame({"token_count": token_count, "type_count": type_count})

print(label_results_train.to_markdown())

|                         |   token_count |   type_count |
|:------------------------|--------------:|-------------:|
| Information/Explanation |        124130 |        14678 |
| News                    |        136557 |        15319 |
| Instruction             |         83750 |         8929 |
| Opinion/Argumentation   |        103141 |        13088 |
| Forum                   |         58900 |         8555 |
| Prose/Lyrical           |         46860 |         5990 |
| Legal                   |         28496 |         4425 |
| Promotion               |         88626 |        12548 |


In [37]:
# Save the label token count
with open("datasets/tokenized_datasets/X-GENRE-train-label-token-count.json", "w") as train_label_count_file:
	json.dump(label_token_count_train, train_label_count_file)

### Create label-level token counts for test sets

In [44]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

pd.DataFrame(main_dict["sl"]["dataset"]).head(2)

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ..."
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414..."


In [46]:
# Do the same as with the dataset, but on every language
lang_results = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")

	label_token_dict_test = {}
	token_count_test = {}
	type_count_test = {}

	# Current df
	df = pd.DataFrame(main_dict[lang]["dataset"])
	display(df.head(2))

	# Create a dictionary of token counts that are label based
	label_token_dict = {}
	token_count = {}
	type_count = {}

	for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
		print(f"Processing {label}")

		label_df = df[df["y_true"] == label]
		display(label_df.head(3))

		# Create a dictionary that counts all the token occurrences

		# Create a list of tokens
		token_list = []

		for i in label_df["tokens"].to_list():
			token_list.extend(i)

		print(f"Number of all tokens: {len(token_list)}")

		# Create a dictionary which counts the occurrences of the words

		word_dict = Counter(token_list)

		# Sort the dictionary alphabetically based on keys
		word_dict = dict(sorted(word_dict.items()))

		print(list(word_dict.items())[:20])
		print(f"Number of all types: {len(list(word_dict.keys()))}")

		# Add to dictionaries
		label_token_dict_test[label] = word_dict
		token_count_test[label] = len(token_list)
		type_count_test[label] = len(list(word_dict.keys()))

	# Add these to a lang-based dict
	lang_results[lang] = {"token_count_dict": label_token_dict_test, "token_count": token_count_test, "type_count": type_count_test }

	# Add this information to the main dictionary
	main_dict[lang]["token_overlap"]["label-level"] = lang_results[lang]


Processing mt


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,macocu.mt.402244,News,"Angelo Chetcuti, se jkun qed jieħu post Bjorn ...","Angelo Chetcuti, will be replacing Bjorn Vassa...",{'text_id': 'macocu.mt.402244'},News,"[▁Angel, o, ▁Che, t, cuti, ,, ▁se, ▁j, kun, ▁q...","[26902, 31, 5024, 18, 64969, 4, 40, 1647, 6262..."
1,macocu.mt.377203,Prose/Lyrical,Poltergeist jirreferi għal fenomeni oħra tal-m...,"Poltergeist refers to other woman's phenomena,...",{'text_id': 'macocu.mt.377203'},Opinion/Argumentation,"[▁Pol, ter, geist, ▁jir, re, feri, ▁g, ħ, al, ...","[9017, 720, 178490, 52826, 107, 26926, 706, 24..."


Processing Information/Explanation


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
8,macocu.mt.241966,Prose/Lyrical,Tales dwar il-magna. Tales Moderna għat-Tfal \...,Tales about the machine.Modern tales for child...,{'text_id': 'macocu.mt.241966'},Information/Explanation,"[▁Tale, s, ▁d, war, ▁il, -, magn, a, ., ▁Tale,...","[59144, 7, 104, 4205, 211, 9, 86049, 11, 5, 59..."
14,macocu.mt.383508,Information/Explanation,L-Isqof ta 'Bridget kien wieħed mid-dsatax-il ...,The Bishop of Bridget was one of the nineteen ...,{'text_id': 'macocu.mt.383508'},Information/Explanation,"[▁L, -, Is, q, of, ▁ta, ▁', B, ridge, t, ▁ki, ...","[339, 9, 29598, 864, 4390, 308, 242, 571, 9185..."
15,macocu.mt.294792,Forum,"Kif tpoġġi ""kelb"" ikona? @ Għaliex huwa msejja...","How to put a ""dog"" icon?@ Why it's called ""dog...",{'text_id': 'macocu.mt.294792'},Information/Explanation,"[▁Ki, f, ▁t, po, ġ, ġ, i, ▁"", kel, b, "", ▁ikon...","[1519, 420, 808, 771, 245673, 245673, 14, 44, ..."


Number of all tokens: 6656
[('"', 9), ('",', 6), ("'", 68), (')', 3), ('),', 2), (').', 4), (',', 193), ('-', 404), ('-12-', 2), ('-16', 3), ('-35', 1), ('-90', 2), ('.', 137), ('."', 1), ('/4', 1), ('12', 2), ('19', 1), ('2.5', 1), ('22', 1), ('25', 1)]
Number of all types: 1446
Processing News


...

Number of all tokens: 2909
[('!', 7), ("''", 1), (')', 2), ('*', 2), ('+', 31), (',', 187), ('.', 93), ('...', 1), ('.«', 1), ('2)', 1), ('25', 1), (':', 8), (':1', 1), (';', 7), ('?', 5), ('Kom', 1), ('M', 1), ('Ma', 1), ('Prav', 1), ('Pri', 1)]
Number of all types: 1131
Processing Legal


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
6,CLASSLA-web.sl.1320205,Legal,JEDRO: Verjetnost obstoja terjatve se presoja ...,Core: The likelihood of the existence of a cla...,"CLASSLA-web.sl.1320205', 'domain': 'sodisce.si'}",Legal,"[▁JE, DRO, :, ▁Ver, jet, nost, ▁obstoj, a, ▁te...","[17754, 122359, 12, 1446, 4249, 3110, 147265, ..."
7,CLASSLA-web.sl.1341965,Legal,JEDRO: V predlogih predstavljene okoliščine na...,Core: In proposals presented circumstances of ...,"CLASSLA-web.sl.1341965', 'domain': 'sodisce.si'}",Legal,"[▁JE, DRO, :, ▁V, ▁predlog, ih, ▁predstavljen,...","[17754, 122359, 12, 310, 68615, 1043, 128967, ..."
31,CLASSLA-web.sl.2114460,Legal,75SUB-EPPO19\nDatum objave:\nUpravičene osebe ...,75SUB-EPPO19 Date of publication: The eligible...,"CLASSLA-web.sl.2114460', 'domain': 'energetika...",Legal,"[▁75, S, UB, -, EP, PO, 19, ▁Datum, ▁objave, :...","[4948, 294, 32338, 9, 21290, 9698, 2947, 76887..."


Number of all tokens: 3446
[('"', 3), ('(1)', 1), (')', 7), ('),', 2), (').', 3), (',', 143), ('-', 7), ('---', 1), ('-1', 1), ('-1)', 1), ('-16', 1), ('.', 106), ('...).', 1), ('.11.', 3), ('.4.', 4), ('.5.', 1), ('/', 5), ('/12', 2), ('/2002', 2), ('/2003', 2)]
Number of all types: 1116
Processing Promotion


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
4,CLASSLA-web.sl.1258381,Promotion,"Umivalniki, ki jih nudimo v naši spletni proda...",The washbasins we offer in our online store ca...,"CLASSLA-web.sl.1258381', 'domain': 'emundia.si'}",Promotion,"[▁Um, ival, niki, ,, ▁ki, ▁jih, ▁nudi, mo, ▁v,...","[2793, 47898, 10926, 4, 200, 5008, 39488, 432,..."
5,CLASSLA-web.sl.1307286,Promotion,Commodore C64\nCommodore USA je začel s prodaj...,The Commodore C64 Commodore USA started sellin...,"CLASSLA-web.sl.1307286', 'domain': 'monitor.si'}",Promotion,"[▁Com, modo, re, ▁C, 64, ▁Com, modo, re, ▁USA,...","[3288, 74445, 107, 313, 13307, 3288, 74445, 10..."
8,CLASSLA-web.sl.1416741,Promotion,Po lepi modri Donavi z Ladjo. Čari starih evro...,After a beautiful blue Danube with a boat. Cha...,"CLASSLA-web.sl.1416741', 'domain': 'mmturist.si'}",Promotion,"[▁Po, ▁lepi, ▁mod, ri, ▁Dona, vi, ▁z, ▁Lad, jo...","[663, 110722, 2811, 416, 100127, 686, 97, 6433..."


Number of all tokens: 3203
[('!', 2), (')', 2), ('),', 2), (').', 1), (',', 127), ('-', 2), ('-20', 1), ('.', 110), ('...', 1), ('/', 1), ('150', 1), ('2', 2), ('64', 7), ('895', 1), ('95', 1), (':', 11), ('A', 2), ('AJ', 1), ('BL', 2), ('C', 3)]
Number of all types: 1371


In [50]:
pd.DataFrame(lang_results["mt"])

,token_count_dict,token_count,type_count
Information/Explanation,"{'""': 9, '"",': 6, ''': 68, ')': 3, '),': 2, ')...",6656,1446
News,"{'""': 3, '"",': 1, ''': 159, ')': 3, ',': 126, ...",6978,1614
Instruction,"{'!': 2, '""': 3, '"",': 1, '"".': 2, ''': 81, ')...",9288,1585
Opinion/Argumentation,"{'!': 4, '""': 1, '"",': 1, ''': 56, ')': 3, '),...",4096,1109
Forum,"{''': 10, ',': 11, '-': 48, '-21': 1, '.': 9, ...",512,240
Prose/Lyrical,"{''': 2, ')': 4, ',': 10, '-': 25, '-24': 1, '...",512,241
Legal,"{'#': 20, ''': 43, '(': 3, '(1)': 4, '(2)': 4,...",5120,1439
Promotion,"{'!': 3, '""': 18, '"",': 2, '"".': 4, ''': 43, '...",5878,1361


### 

In [39]:
# Open label-based train token count
with open("datasets/tokenized_datasets/X-GENRE-train-label-token-count.json", "r") as train_label_count_file:
	label_token_count_train = json.load(train_label_count_file)

label_token_count_train.keys()

dict_keys(['Information/Explanation', 'News', 'Instruction', 'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal', 'Promotion'])

In [40]:
# Import the main dict for test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "r") as file:
	main_dict = json.load(file)

list(main_dict["sl"]["token_overlap"]["token_count"].items())[:10]

[('!', 30),
 ('!!', 1),
 ('!"', 1),
 ('!),', 1),
 ('"', 11),
 ('".', 2),
 ('"...', 1),
 ('&', 8),
 ("'", 3),
 ("''", 1)]

In [ ]:
cosine_sim = {}
vector_size = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")
	# Get token count for current lang
	current_lang_count = main_dict[lang]["token_overlap"]["token_count"]

	print(f"Number of token types for {lang}: {len(list(current_lang_count.keys()))}")

	# For each test set, create a vector of token counts. Take only tokens that are present either in train_df or test set.
	intersection_dict = {}

	# First, create a list of tokens that are present in either one or the other list
	intersection_keys = []
	intersection_keys.extend(list(current_lang_count.keys()))
	intersection_keys.extend(list(train_count.keys()))
	# Remove duplicated keys
	intersection_keys = list(set(intersection_keys))
	print(f"Number of intersecting types: {len(intersection_keys)}")

	# Then create a dictionary for 1) train df and 2) test df with counts of token types that occur in either of the datasets
	train_intersect_dict = {}
	test_intersect_dict = {}

	for i in intersection_keys:
		try:
			train_intersect_dict[i] = train_count[i]
		except:
			train_intersect_dict[i] = 0
		try:
			test_intersect_dict[i] = current_lang_count[i]
		except:
			test_intersect_dict[i] = 0


	# Create a df with intersecting keys
	intersect_df = pd.DataFrame({"train": train_intersect_dict, "test": test_intersect_dict})

	display(intersect_df.head(10))

	# Calculate cosine similarity
	current_cosine_sim = cosine_similarity(np.array(intersect_df["train"].to_list()), np.array(intersect_df["test"].to_list()))

	print(f"Cosine similarity for {lang}: {current_cosine_sim}")

	# Add to the main dictionary
	main_dict[lang]["token_overlap"]["cosine_similarity"] = current_cosine_sim
	main_dict[lang]["token_overlap"]["intersection_df"] = intersect_df.to_dict()
	main_dict[lang]["token_overlap"]["intersection_vector_size"] = len(intersection_keys)

	# Add to a dict of results
	cosine_sim[lang] = current_cosine_sim
	vector_size[lang] = len(intersection_keys)

Processing mt
Number of token types for mt: 4787
Number of intersecting types: 28226


,train,test
▁boot,11,0
▁Trend,2,0
set,47,1
▁skills,38,0
▁ER,2,0
▁Install,1,1
▁Ż,0,7
▁Bran,9,0
NJE,3,0
▁primordial,1,0


Cosine similarity for mt: 0.4142476865826809
Processing el
Number of token types for el: 4751
Number of intersecting types: 30954


,train,test
▁Απριλίου,0,3
▁boot,11,0
▁Trend,2,0
set,47,0
▁skills,38,0
▁ER,2,0
▁Install,1,0
έλθει,0,1
▁Bran,9,0
NJE,3,0


Cosine similarity for el: 0.5273207128330722
Processing tr
Number of token types for tr: 6272
Number of intersecting types: 30846


...

Cosine similarity for mk: 0.4225320009293607
Processing hr
Number of token types for hr: 6222
Number of intersecting types: 28864


,train,test
▁boot,11,0
▁Essen,0,2
▁Trend,2,0
set,47,0
▁skills,38,0
▁ER,2,0
▁Install,1,0
▁Bran,9,0
NJE,3,0
▁primordial,1,0


Cosine similarity for hr: 0.5662098277228971
Processing sl
Number of token types for sl: 5763
Number of intersecting types: 27507


,train,test
▁boot,11,0
▁Trend,2,0
set,47,0
▁skills,38,0
▁ER,2,0
▁Install,1,0
▁Bran,9,0
NJE,3,1
▁primordial,1,0
TION,18,0


Cosine similarity for sl: 0.6334513674796362


In [ ]:
# Show results
cosine_sim_df = pd.DataFrame({"cosine_similarity": cosine_sim, "vector_size": vector_size}).sort_values(by="cosine_similarity", ascending=False)
print(cosine_sim_df.to_markdown())

|    |   cosine_similarity |   vector_size |
|:---|--------------------:|--------------:|
| sl |            0.633451 |         27507 |
| tr |            0.593847 |         30846 |
| uk |            0.584612 |         33121 |
| hr |            0.56621  |         28864 |
| el |            0.527321 |         30954 |
| ca |            0.525439 |         29443 |
| is |            0.525135 |         29518 |
| sq |            0.434489 |         29168 |
| mk |            0.422532 |         31837 |
| mt |            0.414248 |         28226 |


In [ ]:
main_dict["sl"]["token_overlap"].keys()

dict_keys(['overlap_percentage', 'token_list', 'overlap_token_list', 'token_count', 'cosine_similarity', 'intersection_df', 'intersection_vector_size'])

In [ ]:
# Save the main dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)